In [3]:
import pandas as pd
df = pd.read_json('baanknet_property_details.json')
print(df.head())

   status                                           respData  \
0       1  {'residentialDetail': {'propResiDetailId': 151...   
1       1  {'residentialDetail': {'propResiDetailId': 259...   
2       1  {'residentialDetail': {'propResiDetailId': 289...   
3       1  {'residentialDetail': {'propResiDetailId': 202...   
4       1  {'residentialDetail': {'propResiDetailId': 222...   

                                               media  property_id  
0                                               None          209  
1                                               None          351  
2  {'status': 1, 'respMsg': '1003', 'respData': [...          393  
3  {'status': 1, 'respMsg': '1003', 'respData': [...          274  
4  {'status': 1, 'respMsg': '1003', 'respData': [...          300  


In [4]:
# 1. Defining a function to safely extract the nested dictionary
def safe_extract_details(data):
    # First, checking if the data is a dictionary
    if isinstance(data, dict):
        return data.get('residentialDetail')
    # If not (e.g., it's a NaN), returning None
    return None

residential_details_list = df['respData'].apply(safe_extract_details)
cleaned_details_list = residential_details_list.dropna()

# 2. Converting the clean list into a DataFrame.
details_df = pd.DataFrame(cleaned_details_list.tolist(), index=cleaned_details_list.index)

# 3. Joining the new details_df with the original 'property_id' column
final_df = df[['property_id']].join(details_df)

# Now, dropping any rows from the final DataFrame that are still empty after the join
final_df.dropna(subset=details_df.columns, inplace=True)


# result
print("Cleaned and Flattened DataFrame:")
print(final_df.head())

print("\nNew DataFrame Info:")
final_df.info()

Cleaned and Flattened DataFrame:
Empty DataFrame
Columns: [property_id, propResiDetailId, propertyDetailId, carpetAreaSqft, builtupAreaSqft, furnishedStatus, ageOfConstruction, priceSqfeet, floorno, totalNoOfFloor, unitOnFloor, isCornerProperty, waterAvailability, electriCityStatus, noOfRoom, nearByEduInst, nearByShopCent, nearByLocality, nearByCommHub, noOfCarParking, developerName, projectName, facing, otherDetail, washRoom, masterUnitOfMeasure]
Index: []

[0 rows x 26 columns]

New DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   property_id          0 non-null      int64  
 1   propResiDetailId     0 non-null      float64
 2   propertyDetailId     0 non-null      float64
 3   carpetAreaSqft       0 non-null      float64
 4   builtupAreaSqft      0 non-null      float64
 5   furnishedStatus      0 non-null      object 
 6   a

In [5]:
# to see data types and non-null counts for the new columns
final_df.info()

# Checking for any missing values in the new columns
print("\nMissing values:")
print(final_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   property_id          0 non-null      int64  
 1   propResiDetailId     0 non-null      float64
 2   propertyDetailId     0 non-null      float64
 3   carpetAreaSqft       0 non-null      float64
 4   builtupAreaSqft      0 non-null      float64
 5   furnishedStatus      0 non-null      object 
 6   ageOfConstruction    0 non-null      float64
 7   priceSqfeet          0 non-null      object 
 8   floorno              0 non-null      object 
 9   totalNoOfFloor       0 non-null      float64
 10  unitOnFloor          0 non-null      object 
 11  isCornerProperty     0 non-null      object 
 12  waterAvailability    0 non-null      object 
 13  electriCityStatus    0 non-null      object 
 14  noOfRoom             0 non-null      object 
 15  nearByEduInst        0 non-null      object 
 16  nearByS

In [6]:
# printing a list of all column names
print(final_df.columns)

Index(['property_id', 'propResiDetailId', 'propertyDetailId', 'carpetAreaSqft',
       'builtupAreaSqft', 'furnishedStatus', 'ageOfConstruction',
       'priceSqfeet', 'floorno', 'totalNoOfFloor', 'unitOnFloor',
       'isCornerProperty', 'waterAvailability', 'electriCityStatus',
       'noOfRoom', 'nearByEduInst', 'nearByShopCent', 'nearByLocality',
       'nearByCommHub', 'noOfCarParking', 'developerName', 'projectName',
       'facing', 'otherDetail', 'washRoom', 'masterUnitOfMeasure'],
      dtype='object')


In [7]:
import pandas as pd

# First, ensure the columns you need are numeric. 'coerce' will make non-numeric entries NaN.
final_df['priceSqfeet'] = pd.to_numeric(final_df['priceSqfeet'], errors='coerce')
final_df['builtupAreaSqft'] = pd.to_numeric(final_df['builtupAreaSqft'], errors='coerce')

# It's important to drop rows where these essential values are missing
final_df.dropna(subset=['priceSqfeet', 'builtupAreaSqft'], inplace=True)

# Now, create your target variable 'total_price'
final_df['total_price'] = final_df['priceSqfeet'] * final_df['builtupAreaSqft']

print("Successfully created the 'total_price' column.")
print(final_df[['priceSqfeet', 'builtupAreaSqft', 'total_price']].head())

Successfully created the 'total_price' column.
Empty DataFrame
Columns: [priceSqfeet, builtupAreaSqft, total_price]
Index: []


In [8]:
print("--- Debugging Info ---")

# Checking the data types and how many non-missing values exist
print("Info for price and area columns:")
final_df[['priceSqfeet', 'builtupAreaSqft', 'carpetAreaSqft']].info()

# Looking at the first 10 rows just to see what the data actually looks like
print("\nSample of the data:")
print(final_df[['priceSqfeet', 'builtupAreaSqft', 'carpetAreaSqft']].head(10))

# Seeing the most common values in the columns
print("\nMost common values in 'builtupAreaSqft':")
print(final_df['builtupAreaSqft'].value_counts().head())

--- Debugging Info ---
Info for price and area columns:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   priceSqfeet      0 non-null      int64  
 1   builtupAreaSqft  0 non-null      float64
 2   carpetAreaSqft   0 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 0.0 bytes

Sample of the data:
Empty DataFrame
Columns: [priceSqfeet, builtupAreaSqft, carpetAreaSqft]
Index: []

Most common values in 'builtupAreaSqft':
Series([], Name: count, dtype: int64)


In [9]:
print("--- Inspecting the raw 'respData' column ---")

#Checking if the column has data at all
print(f"Total rows: {len(df)}")
print(f"Missing values in 'respData': {df['respData'].isnull().sum()}")

#Finding the first non-empty entry to inspect the structure
first_valid_entry = df['respData'].dropna().iloc[0]
print("\nStructure of the first valid entry:")
print(first_valid_entry)

#Printing the keys of that dictionary to find the right one
print("\n>>> Keys available in this dictionary:")
print(first_valid_entry.keys())

--- Inspecting the raw 'respData' column ---
Total rows: 101
Missing values in 'respData': 20

Structure of the first valid entry:
{'residentialDetail': {'propResiDetailId': 151, 'propertyDetailId': 209, 'carpetAreaSqft': None, 'builtupAreaSqft': None, 'furnishedStatus': None, 'ageOfConstruction': None, 'priceSqfeet': None, 'floorno': None, 'totalNoOfFloor': None, 'unitOnFloor': None, 'isCornerProperty': None, 'waterAvailability': None, 'electriCityStatus': None, 'noOfRoom': None, 'nearByEduInst': None, 'nearByShopCent': None, 'nearByLocality': None, 'nearByCommHub': None, 'noOfCarParking': None, 'developerName': None, 'projectName': None, 'facing': None, 'otherDetail': None, 'washRoom': None, 'masterUnitOfMeasure': None}, 'commercialDetail': None, 'agricultureDetail': None, 'industryDetail': None, 'otherDetail': None, 'propertyAmenitieslist': None, 'masterPropertyType': 1, 'propertyType': 'Individual House', 'summaryDesc': 'LAND AND BUILDING AT PLOT NO.95, TS NO.35 PART OF PULIYUR VIL

In [10]:
import pandas as pd
data_list = df['respData'].dropna().tolist()
final_df = pd.json_normalize(data_list)
print("Success! A new, rich DataFrame has been created.")

pd.set_option('display.max_columns', None) #
print("\nFirst 5 rows of the new DataFrame:")
print(final_df.head())

print("\nAvailable columns:")
print(final_df.columns)

Success! A new, rich DataFrame has been created.

First 5 rows of the new DataFrame:
  commercialDetail agricultureDetail industryDetail otherDetail  \
0             None              None           None        None   
1             None              None           None        None   
2             None              None           None        None   
3             None              None           None        None   
4             None              None           None        None   

  propertyAmenitieslist  masterPropertyType      propertyType  \
0                  None                   1  Individual House   
1                  None                   1              Plot   
2                  None                   1  Individual House   
3                  None                   1  Individual House   
4                  None                   1  Individual House   

                                         summaryDesc  propertyPrice  \
0  LAND AND BUILDING AT PLOT NO.95, TS NO.35 PART.

In [11]:
# Calculating the percentage of missing values for each column
missing_percentage = final_df.isnull().sum() / len(final_df) * 100

# Set a threshold for missing values
threshold = 50  # Dropping columns with more than 50% missing data

# Getting the list of columns to drop
cols_to_drop = missing_percentage[missing_percentage > threshold].index

# Dropping these columns from the DataFrame
cleaned_df = final_df.drop(columns=cols_to_drop)

print(f"Original number of columns: {len(final_df.columns)}")
print(f"Columns remaining after dropping empty ones: {len(cleaned_df.columns)}")

print("\nRemaining columns:")
print(cleaned_df.columns)

Original number of columns: 178
Columns remaining after dropping empty ones: 89

Remaining columns:
Index(['masterPropertyType', 'propertyType', 'summaryDesc', 'propertyPrice',
       'city', 'bankName', 'bankPropertyId', 'isFavProperty', 'visitOrCount',
       'propertyPhoto', 'imgCount', 'residentialDetail.propResiDetailId',
       'residentialDetail.propertyDetailId',
       'commonPropertyDetails.propertyDetailId',
       'commonPropertyDetails.department.departmentId',
       'commonPropertyDetails.department.clientId',
       'commonPropertyDetails.department.departmentName',
       'commonPropertyDetails.department.parentDepartmentId',
       'commonPropertyDetails.department.officeTypeId',
       'commonPropertyDetails.department.bank.bankId',
       'commonPropertyDetails.department.bank.ifsc',
       'commonPropertyDetails.department.bank.bankCode',
       'commonPropertyDetails.department.bank.bankLogo',
       'commonPropertyDetails.department.bank.bankName',
       'common

In [12]:
#list of promising column names based on the output above
feature_columns = [
    'propertyPrice',
    'propertyType',
    'city',
    'visitOrCount',
    'auctionDetails.EMD'
]


# Selecting only the columns that exist in your cleaned_df
# This avoids errors
final_cols = [col for col in feature_columns if col in cleaned_df.columns]

#final DataFrame for modeling
model_df = cleaned_df[final_cols]

# Renaming columns for easier access
model_df = model_df.rename(columns={
    'auctionDetails.EMD': 'emd',
    'auctionDetails.ReservePrice': 'reserve_price',
    'commonPropertyDetails.pincode': 'pincode',
    'commonPropertyDetails.npaAmount': 'npa_amount'
})


print("\nFinal DataFrame for modeling:")
print(model_df.head())

print("\nFinal DataFrame Info:")
model_df.info()


Final DataFrame for modeling:
   propertyPrice      propertyType           city  visitOrCount
0      7545000.0  Individual House        Chennai           152
1      1220000.0              Plot          Olpad            17
2      3800000.0  Individual House  Neyyattinkara            15
3      2578000.0  Individual House      Firozabad            27
4      2524000.0  Individual House          Bhind            25

Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   propertyPrice  81 non-null     float64
 1   propertyType   81 non-null     object 
 2   city           81 non-null     object 
 3   visitOrCount   81 non-null     int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 2.7+ KB


In [13]:
import pandas as pd
model_df_encoded = pd.get_dummies(model_df, drop_first=True)

print("DataFrame shape after encoding:", model_df_encoded.shape)
print(model_df_encoded.head())

DataFrame shape after encoding: (81, 64)
   propertyPrice  visitOrCount  propertyType_Flat  \
0      7545000.0           152              False   
1      1220000.0            17              False   
2      3800000.0            15              False   
3      2578000.0            27              False   
4      2524000.0            25              False   

   propertyType_Individual House  propertyType_Plot  propertyType_Villa  \
0                           True              False               False   
1                          False               True               False   
2                           True              False               False   
3                           True              False               False   
4                           True              False               False   

   city_Ahmedabad  city_Aizawl  city_Aranthangi  city_Bahadurpur  city_Barkur  \
0           False        False            False            False        False   
1           False        Fa

In [20]:

feature_columns = [
    'propertyPrice',
    'propertyType',
    'city',
    'visitOrCount'
]


#final modeling dataframe
model_df = final_df[feature_columns]

# Renaming columns for easier use
model_df = model_df.rename(columns={'commonPropertyDetails.pincode': 'pincode',
                                    'auctionDetails.EMD': 'emd'})

print("\nFinal DataFrame for modeling:")
print(model_df.head())


Final DataFrame for modeling:
   propertyPrice      propertyType           city  visitOrCount
0      7545000.0  Individual House        Chennai           152
1      1220000.0              Plot          Olpad            17
2      3800000.0  Individual House  Neyyattinkara            15
3      2578000.0  Individual House      Firozabad            27
4      2524000.0  Individual House          Bhind            25


In [19]:
from sklearn.model_selection import train_test_split

X = model_df_encoded.drop('propertyPrice', axis=1)
y = model_df_encoded['propertyPrice']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Training set shape: (64, 63)
Testing set shape: (17, 63)


In [21]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


ridge_model = Ridge(alpha=1.0) # as alpha controls the strength of regularization
print("\nTraining a simpler Ridge model...")
ridge_model.fit(X_train_scaled, y_train)

y_pred_ridge = ridge_model.predict(X_test_scaled)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("\n--- Ridge Model Evaluation ---")
print(f"Mean Absolute Error (MAE): ₹{mae_ridge:,.2f}")
print(f"R-squared (R²) Score: {r2_ridge:.2f}")


Training a simpler Ridge model...

--- Ridge Model Evaluation ---
Mean Absolute Error (MAE): ₹3,998,154.97
R-squared (R²) Score: -0.13


In [22]:
import joblib

# --- Save the Model and Scaler ---

# 1. Define the filenames
model_filename = 'ridge_model.pkl'
scaler_filename = 'scaler.pkl'

# 2. Save the objects to files
joblib.dump(ridge_model, model_filename)
joblib.dump(scaler, scaler_filename)

print(f"\nModel saved to: {model_filename}")
print(f"Scaler saved to: {scaler_filename}")


Model saved to: ridge_model.pkl
Scaler saved to: scaler.pkl
